In [1]:
%load_ext autoreload
%autoreload 2
import os
from tqdm import tqdm

from histopolalign.CompareHealthyTumor import compare_healthy_tumor
from histopolalign.VerificationAlignment import mask_generation

In [2]:
type_ = ''

## 1. Healthy polarimetric parameters

#### 1.1. Process the Mueller matrices, if needed

In [3]:
healthy_polarimetry_path = os.path.join(os.getcwd().split('notebooks')[0], 'data', 'HealthyHuman')
directories = [healthy_polarimetry_path]
calib_directory = os.path.join(os.getcwd().split('notebooks')[0], 'calib')

# from processingmm import batch_processing
# batch_processing.batch_process(directories, calib_directory)

#### 1.2. Get the grey / white matter masks

In [4]:
folders_masks = []
for folder in os.listdir(healthy_polarimetry_path):
    if type_ in folder:
        folders_masks.append(os.path.join(healthy_polarimetry_path, folder))

_ = mask_generation.create_the_masks(folders_of_interest = folders_masks)

100%|████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:12<00:00,  1.60s/it]


#### 1.3. Get the values for healthy tissue

In [5]:
azimuth_sq_size = 4
values_healthy = compare_healthy_tumor.get_parameters_healthy(healthy_polarimetry_path, azimuth_sq_size = azimuth_sq_size, type_ = type_)

100%|████████████████████████████████████████████████████████████████████████████████████| 8/8 [01:16<00:00,  9.59s/it]


## 2. Neoplastic polarimetric parameters

#### 2.1. Get the values for neoplastic tissues

In [6]:
neoplastic_polarimetry_path = os.path.join(os.getcwd().split('notebooks')[0], 'data', 'TumorMeasurements')
path_folders = compare_healthy_tumor.get_all_folders(neoplastic_polarimetry_path, type_ = type_)
azimuth_sq_size = 4
values_folder = compare_healthy_tumor.get_the_values(path_folders, azimuth_sq_size = azimuth_sq_size)

100%|██████████████████████████████████████████████████████████████████████████████████| 44/44 [06:43<00:00,  9.17s/it]


#### 2.2. And plot the histograms for the paper

In [7]:
parameters = ['totP', 'linR', 'azimuth']
compare_healthy_tumor.plot_histograms_paper(values_healthy, values_folder, parameters, (153, 153, 0),
                                os.path.join(os.getcwd().split('notebooks')[0], 'results', 'WM_split' + type_ + '.pdf'), WM = True)
compare_healthy_tumor.plot_histograms_paper(values_healthy, values_folder, parameters, (153, 77, 0),
                                os.path.join(os.getcwd().split('notebooks')[0], 'results', 'GM_split' + type_ + '.pdf'), WM = False)

C:\Users\romai\.conda\envs\pyimagej\lib\site-packages\numpy\lib\histograms.py:906: RuntimeWarning: invalid value encountered in divide
  return n/db/n.sum(), bin_edges
C:\Users\romai\Documents\PathologyPaper\src\histopolalign\CompareHealthyTumor\compare_healthy_tumor.py:291: RuntimeWarning: Mean of empty slice
  mean = np.nanmean(values[idx_param])
C:\Users\romai\.conda\envs\pyimagej\lib\site-packages\numpy\lib\nanfunctions.py:1878: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
C:\Users\romai\.conda\envs\pyimagej\lib\site-packages\numpy\lib\nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
C:\Users\romai\.conda\envs\pyimagej\lib\site-packages\numpy\lib\histograms.py:906: RuntimeWarning: invalid value encountered in divide
  return n/db/n.sum(), bin_edges
C:\Users\romai\Documents\PathologyPaper\src\histopolalign\CompareHealthyTumor\compare_healthy_tumor.py:

#### 2.3. Combine the values across the different tumor cell contents

In [8]:
values_folder_combined = compare_healthy_tumor.combine_the_values(values_folder)

compare_healthy_tumor.plot_histograms_paper_combined(values_healthy, values_folder_combined, parameters, False,
                               os.path.join(os.getcwd().split('notebooks')[0], 'results', 'GM_combined.pdf'))
compare_healthy_tumor.plot_histograms_paper_combined(values_healthy, values_folder_combined, parameters, True,
                               os.path.join(os.getcwd().split('notebooks')[0], 'results', 'WM_combined.pdf'))

In [9]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.optimize import curve_fit
from scipy.stats import exponnorm
import scipy

def gauss(x, mu, sigma, A):
    return A*np.exp(-(x-mu)**2/2/sigma**2)

def bimodal(x, mu1, sigma1, A1, mu2, sigma2, A2):
    return gauss(x,mu1, sigma1, A1)+gauss(x,mu2, sigma2, A2)

    
def fit_data_healthy(values, color, ax, expected = [16, 4, 1, 32, 10, 0.3]):

    #data generation
    data = np.array(values[255])
    y, x = np.histogram(data, bins=np.linspace(0, 80, 81))
    x=(x[1:]+x[:-1])/2
    y = y/np.max(y)
    ax.bar(x, y,width=1, color=color, alpha = 0.3)
    params, cov = curve_fit(bimodal, x, y, expected)
    sigma=np.sqrt(np.diag(cov))
    x_fit = np.linspace(x.min(), x.max(), 500)
    ax.plot(x_fit, bimodal(x_fit, *params), color=color, lw=2.5, ls="-", label='Model')
    ax.plot(x_fit, gauss(x_fit, *params[:3]), color=color, lw=2.5, ls=":", label='Gaussian 1')
    ax.plot(x_fit, gauss(x_fit, *params[3:]), color=color, lw=2.5, ls="--", label='Gaussian 2')
    ax.legend()
    ax.set_xlim([0, 80])
    ax.set_ylim([0, 1.5])

    ax.legend()
    
    ax.xaxis.set_major_locator(plt.MaxNLocator(4))
    ax.yaxis.set_major_locator(plt.MaxNLocator(3))

    # change the font size of the ticks
    ax.tick_params(axis='both', which='major', labelsize=18)

    for tick in ax.xaxis.get_major_ticks():
        tick.label1.set_fontsize(12)
        tick.label1.set_fontweight('bold')
    for tick in ax.yaxis.get_major_ticks():
        tick.label1.set_fontsize(12)
        tick.label1.set_fontweight('bold')
            
    return pd.DataFrame(data={'params': params, 'sigma': sigma}, index=bimodal.__code__.co_varnames[1:])


def fit_data(values, color, ax, expected = (16, 4, 1, 31, 15, 1)):
    data = np.array(values)
    
    #data generation
    y, x = np.histogram(values, bins=np.linspace(0, 80, 81))
    x=(x[1:]+x[:-1])/2
    y = y/np.max(y)
    ax.bar(x, y,width=1, color=color, alpha = 0.3)
    
    params, cov = curve_fit(bimodal, x, y, expected)
    sigma=np.sqrt(np.diag(cov))
    x_fit = np.linspace(x.min(), x.max(), 500)
    #plot combined...
    #...and individual Gauss curves
    ax.plot(x_fit, bimodal(x_fit, *params), color=color, lw=2.5, ls="-", label='Model')
    ax.plot(x_fit, gauss(x_fit, *params[:3]), color=color, lw=2.5, ls=":", label='Gaussian 1')
    ax.plot(x_fit, gauss(x_fit, *params[3:]), color=color, lw=2.5, ls="--", label='Gaussian 2')
    ax.set_xlim([0, 80])
    ax.set_ylim([0, 1.20])
    
    ax.xaxis.set_major_locator(plt.MaxNLocator(4))
    ax.yaxis.set_major_locator(plt.MaxNLocator(3))

    # change the font size of the ticks
    ax.tick_params(axis='both', which='major', labelsize=18)
    ax.set_yticks([0, 0.25, 0.50, 0.75, 1])
    
    for tick in ax.xaxis.get_major_ticks():
        tick.label1.set_fontsize(12)
        tick.label1.set_fontweight('bold')
    for tick in ax.yaxis.get_major_ticks():
        tick.label1.set_fontsize(12)
        tick.label1.set_fontweight('bold')
            
    return pd.DataFrame(data={'params': params, 'sigma': sigma}, index=bimodal.__code__.co_varnames[1:])

values = values_folder['azimuth'][(153, 153, 0)]

def fit_all_data(values, values_healthy):
    fig, axs = plt.subplots(2, 2, layout='constrained', figsize=(10, 8))
    parameters_TF = fit_data(values_healthy['azimuth'][255], color = 'green', ax = axs[0][0], expected = [16, 4, 1, 32, 10, 0.3])
    parameters_LLI = fit_data(values[(0, 255, 255)], color = 'turquoise', ax = axs[0][1], expected = [10, 10, 1, 32, 10, 1])
    # parameters_HLI = fit_data(values[(0, 0, 255)], color = 'blue', ax = axs[1][0])
    parameters_TC = fit_data(values[(255, 0, 0)], color = 'red', ax = axs[1][1])
    plt.tight_layout()
    return parameters_TF, parameters_LLI, parameters_TC# parameters_HLI, parameters_TC


In [10]:
parameters = fit_all_data(values, values_healthy)
plt.savefig(os.path.join(os.getcwd().split('notebooks')[0], 'results', 'gaussian_fits.pdf'))
plt.close()

C:\Users\romai\AppData\Local\Temp\ipykernel_24956\718302831.py:95: UserWarning: The figure layout has changed to tight
  plt.tight_layout()


In [11]:
parameters

(           params     sigma
 mu1      7.021029  0.168294
 sigma1   2.761822  0.267528
 A1       0.733902  0.056905
 mu2     19.756927  1.475371
 sigma2  13.462711  1.171206
 A2       0.459435  0.018675,
            params     sigma
 mu1     10.299671  0.246534
 sigma1   4.597449  0.362087
 A1       0.420115  0.030807
 mu2     31.048691  0.372897
 sigma2  14.439597  0.344792
 A2       1.009861  0.010590,
            params     sigma
 mu1     11.884100  0.560778
 sigma1  -3.788374  0.606499
 A1       0.053969  0.006979
 mu2     37.035978  0.062156
 sigma2  12.476747  0.069654
 A2       0.981041  0.003726)

## 3. Get the values for the methods of the paper

In [12]:
polarimetry_path_methods = os.path.join(os.getcwd().split('notebooks')[0], 'data', 'HealthyMethods')

azimuth_sq_size = 4

folders_masks = []
for folder in os.listdir(polarimetry_path_methods):
    folders_masks.append(os.path.join(polarimetry_path_methods, folder))

_ = mask_generation.create_the_masks(folders_of_interest = folders_masks)
    
values_healthy_methods = compare_healthy_tumor.get_parameters_healthy(polarimetry_path_methods, 
                                                                      azimuth_sq_size = azimuth_sq_size)

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:10<00:00, 10.37s/it]


In [13]:
neoplastic_polarimetry_path = os.path.join(os.getcwd().split('notebooks')[0], 'data', 'TumorMeasurementsMethods')
path_folders = compare_healthy_tumor.get_all_folders(neoplastic_polarimetry_path)
values_folder = compare_healthy_tumor.get_the_values(path_folders, azimuth_sq_size = azimuth_sq_size)

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:08<00:00,  8.97s/it]


In [14]:
parameters = ['totP', 'linR', 'azimuth']

compare_healthy_tumor.plot_histograms_methods(values_healthy_methods, values_folder, parameters, True,
                      os.path.join(os.getcwd().split('notebooks')[0], 'results', 'methods.pdf'))